# **CS 4361/5361 Machine Learning**

**Linear Regression**

**Author:** [Olac Fuentes](http://www.cs.utep.edu/ofuentes/)<br>
**Last modified:** 2021/09/30<br>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.metrics import accuracy_score, confusion_matrix,mean_squared_error,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree
from google.colab import files

In [ ]:
uploaded = files.upload()

In [ ]:
df = pd.read_csv('gpu_running_time.csv')
df

In [ ]:
data = df.to_numpy()
X = data[:,:14]
y = np.mean(data[:,14:],axis=1)
feature_names = df.columns
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4361)

In [ ]:
b = np.mean(y_train)
W = np.matmul(np.linalg.pinv(X_train),(y_train-np.mean(y_train)).reshape(-1,1))
W.shape

In [ ]:
pred = np.matmul(X_test,W) + b
print('Mean squared error = {:5.2f}'.format(mean_squared_error(pred,y_test)))
print('Mean absolute error =  {:5.2f}'.format(mean_absolute_error(pred,y_test)))

Now, using the sklearn implementation:

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
model.fit(X_train,y_train)
pred = model.predict(X_test)
print('Mean squared error = {:5.2f}'.format(mean_squared_error(pred,y_test)))
print('Mean absolute error =  {:5.2f}'.format(mean_absolute_error(pred,y_test)))

In [ ]:
pred = model.predict(X_train)
print('Mean squared error = {:5.2f}'.format(mean_squared_error(pred,y_train)))
print('Mean absolute error =  {:5.2f}'.format(mean_absolute_error(pred,y_train)))

Let's consider a synthetic dataset implementing the function y = 3x + 5, with some noise added.

In [ ]:
x = np.random.uniform(low=-5,high=15,size =(200,1))
y = 3*x + 5
y = y + np.random.normal(scale=5, size =(200,1)) 
plt.plot(x,y,'.')

Let's see if sklearn can learn the function.

In [ ]:
model = LinearRegression()
model.fit(x,y)
pred = model.predict(x)
print('Mean squared error = {:5.2f}'.format(mean_squared_error(pred,y)))
print('Mean absolute error =  {:5.2f}'.format(mean_absolute_error(pred,y)))

The parameters learned by the model are:

In [ ]:
print(model.coef_)
print(model.intercept_)

where model.coef_ is the array W and model.intercept_ is the scalar b in our equations above.

Now let's plot what the model learned (in orange) compared with the original data (in blue). 

In [ ]:
plt.plot(x,y,'.')
plt.plot(x,pred,'.')

Now let's try to learn a quadartic function.

y = 2x^2 - 5x + 3

In [ ]:
x = np.random.uniform(low=-5,high=15,size =(200,1))
y = 2*x**2 - 5*x + 3
y = y + np.random.normal(scale=5, size =(200,1)) 
plt.plot(x,y,'.')

In [ ]:
model = LinearRegression()
model.fit(x,y)
pred = model.predict(x)
print('Mean squared error = {:5.2f}'.format(mean_squared_error(pred,y)))
print('Mean absolute error =  {:5.2f}'.format(mean_absolute_error(pred,y)))

print('Model parameters:')
print(model.coef_)
print(model.intercept_)

plt.plot(x,y,'.')
plt.plot(x,pred,'.')

Performance is bad using the original features. We can add x^2 as an extra feature.

In [ ]:
model = LinearRegression()
xx = np.hstack((x,x**2))
model.fit(xx,y)
pred = model.predict(xx)
print('Mean squared error = {:5.2f}'.format(mean_squared_error(pred,y)))
print('Mean absolute error =  {:5.2f}'.format(mean_absolute_error(pred,y)))

print('Model parameters:')

print(model.coef_)
print(model.intercept_)

plt.plot(x,y,'.')
plt.plot(x,pred,'.')

Performance is much better.

Since in practice we don't know what terms y depends on, we can add the terms that we think MIGHT be useful, hoping the algorithm will figure out which ones are important.

Let's include x^2, x^3 and x^4 terms. 

In [ ]:
model = LinearRegression()
xx = np.hstack((x,x**2,x**3,x**4))
model.fit(xx,y)
pred = model.predict(xx)
print('Mean squared error = {:5.2f}'.format(mean_squared_error(pred,y)))
print('Mean absolute error =  {:5.2f}'.format(mean_absolute_error(pred,y)))

print('Model parameters:')

print(model.coef_)
print(model.intercept_)

plt.plot(x,y,'.')
plt.plot(x,pred,'.')

Performance is still good and the algorithm assigns small weights to the irrelevant features. 

**Exercises:**

1.  Add quadratic features to the gpu dataset and evaluate the performance of linear regression
2. Use a one-hot representation to classify the MNIST dataset using linear regression



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4361)
for i in range(X.shape[1]):
  for j in range(i, X.shape[1]):
    X_train = np.hstack((X_train, (X_train[:,i]*X_train[:,j]).reshape(-1,1)))
    X_test = np.hstack((X_test, (X_test[:,i]*X_test[:,j]).reshape(-1,1)))
  print(X_train.shape)

A faster implementation using broadcasting:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4361)
for i in range(X.shape[1]):
  X_train = np.hstack((X_train, (X_train[:,i:i+1]*X_train[:,i:X.shape[1]])))
  X_test = np.hstack((X_test, (X_test[:,i:i+1]*X_test[:,i:X.shape[1]])))
  print(X_train.shape)

In [ ]:
model = LinearRegression()
model.fit(X_train,y_train)
pred = model.predict(X_test)
print('Mean squared error = {:5.2f}'.format(mean_squared_error(pred,y_test)))
print('Mean absolute error =  {:5.2f}'.format(mean_absolute_error(pred,y_test)))

In [ ]:
def onehot(y):
  y_oh = np.zeros((len(y), np.amax(y)+1), dtype=int)
  y_oh[np.arange(len(y)), y]=1
  return y_oh

In [ ]:
def onehot_to_class(y_oh):
  return np.argmax(y_oh,axis=1)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = np.float32(x_train/255).reshape(x_train.shape[0],-1)
x_test = np.float32(x_test/255).reshape(x_test.shape[0],-1)

In [ ]:
model = LinearRegression()
model.fit(x_train,onehot(y_train))
pred = onehot_to_class(model.predict(x_test))

print('Accuracy = {:7.4f}'.format(accuracy_score(pred,y_test)))